In [109]:
import sys
sys.path.append("../../src/")
import numpy as np
import math
import uncertainpy.gradual as grad
import random
random.seed(0)

In [110]:
# generate random sublist from current list
def generate_new_list(original_list):
    new_list = []
    for element in original_list:
        if random.choice([True, False]):
            new_list.append(element)
    return new_list

In [111]:
# return cur_arg's SHAP AAE
def compute_SHAP(bag, topic_a, cur_arg, agg_f, inf_f):
    bag = grad.BAG("../../bags/movie_recommender.bag")

    strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)

    arg_name_all = []

    topic_arg = bag.arguments[topic_a]
    for arg in bag.arguments.values():
        arg_name_all.append(arg.name)
        # print((arg.name,arg.strength))
    arg_name_no_topic_arg = arg_name_all.copy()
    arg_name_no_topic_arg.remove(topic_a)

    print(arg_name_all)
    print(arg_name_no_topic_arg)

    # remove current argument
    arg_name_no_topic_no_cur = arg_name_no_topic_arg.copy()
    arg_name_no_topic_no_cur.remove(cur_arg)
    print(arg_name_no_topic_no_cur)
    print("----------------------------")


    # record initial base scores
    arg_initial_weight = []
    for arg in bag.arguments.values():
        arg_initial_weight.append((arg.name,arg.initial_weight))


    # print all the initial base scores
    print("All the base score and strength in QBAF:")
    for arg in bag.arguments.values():
        print((arg.name,arg.initial_weight,arg.strength))


    sample_count = 1000 # iterate 1000 times
    difference = []
    for i in range(sample_count):

        new_list = generate_new_list(arg_name_no_topic_no_cur) # choose random elements in arg_name_no_topic_arg
        print(f"new_list:{new_list}")

        if new_list == []:
            new_tuple = ()
        else:
            new_tuple = tuple(new_list)
        j = new_tuple


        # remove arg(j) from the current combination i
        for k in j: # first remove the argument from the combination
            bag.remove_argument(k)

        strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)

        # compute sigma(S U {i})
        print("Initial and final strength (1st):")
        for arg in bag.arguments.values():
            print((arg.name,arg.initial_weight,arg.strength))
        topic_arg = bag.arguments[topic_a]
        strength_before_drop_cur = topic_arg.strength
        print(f"strength_before_drop_cur:{strength_before_drop_cur}")

        # remove cur arg
        bag.remove_argument(cur_arg)
        strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)

        # compute sigma(S)
        print("Initial and final strength (2nd):")
        for arg in bag.arguments.values():
            print((arg.name,arg.initial_weight,arg.strength))
        topic_arg = bag.arguments[topic_a]
        strength_after_drop_cur = topic_arg.strength
        print(f"strength_after_drop_cur:{strength_after_drop_cur}")

        # compute sigma(S U {i})-sigma(S)
        difference.append(strength_before_drop_cur-strength_after_drop_cur)
        print(f"diff={strength_before_drop_cur-strength_after_drop_cur}")


        # recover BAG for the next iteration
        bag = grad.BAG("../../bags/movie_recommender.bag")
        strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)


    print("----------------------------")

    print(f"difference:{difference}")
    attribution = sum(difference)/sample_count
    print(attribution)

    # reset to original initial weight
    bag = grad.BAG("../../bags/movie_recommender.bag")
    strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)

    return attribution

In [112]:
# Computation via Forward Propagation
bag = grad.BAG("../../bags/movie_recommender.bag")

# # QE
# agg_f = grad.semantics.modular.SumAggregation()
# inf_f = grad.semantics.modular.QuadraticMaximumInfluence(conservativeness=1)

# DF-QUAD
agg_f = grad.semantics.modular.ProductAggregation()
inf_f = grad.semantics.modular.LinearInfluence(conservativeness=1)

# # SD-DF-QUAD
# agg_f = grad.semantics.modular.ProductAggregation()
# inf_f = grad.semantics.modular.SQDFQUADInfluence(conservativeness=1)

# # EB
# agg_f = grad.semantics.modular.SumAggregation()
# inf_f = grad.semantics.modular.EulerBasedInfluence()

# # EBT
# agg_f = grad.semantics.modular.TopAggregation()
# inf_f = grad.semantics.modular.EulerBasedInfluence()


strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)

In [113]:
# compute attribution for one arg
topic_a = "m"
all_args = bag.get_arguments()
all_args_names = []
for i in all_args:
    all_args_names.append(i.get_name())
print(all_args_names)

all_args_names_without_topic = all_args_names.copy()
all_args_names_without_topic.remove(topic_a)
print(all_args_names_without_topic)

attribution = {}
for cur_arg in all_args_names_without_topic:
    attribution[cur_arg] = compute_SHAP(bag, topic_a, cur_arg, agg_f, inf_f)
print(f"attribution: \n{attribution}")

['m', 'A', '1', '2', 'D', 'W']
['A', '1', '2', 'D', 'W']
['m', 'A', '1', '2', 'D', 'W']
['A', '1', '2', 'D', 'W']
['1', '2', 'D', 'W']
----------------------------
All the base score and strength in QBAF:
('m', 0.79, 0.84774307)
('A', 0.16, 0.25786000000000003)
('1', 0.05, 0.05)
('2', 0.07, 0.07)
('D', 0.05, 0.05)
('W', 0.02, 0.02)
new_list:['D']
Initial and final strength (1st):
('m', 0.79, 0.8399506)
('A', 0.16, 0.25786000000000003)
('1', 0.05, 0.05)
('2', 0.07, 0.07)
('W', 0.02, 0.02)
strength_before_drop_cur:0.8399506
Initial and final strength (2nd):
('m', 0.79, 0.7742)
('1', 0.05, 0.05)
('2', 0.07, 0.07)
('W', 0.02, 0.02)
strength_after_drop_cur:0.7742
diff=0.06575059999999999
new_list:[]
Initial and final strength (1st):
('m', 0.79, 0.84774307)
('A', 0.16, 0.25786000000000003)
('1', 0.05, 0.05)
('2', 0.07, 0.07)
('D', 0.05, 0.05)
('W', 0.02, 0.02)
strength_before_drop_cur:0.84774307
Initial and final strength (2nd):
('m', 0.79, 0.7963)
('1', 0.05, 0.05)
('2', 0.07, 0.07)
('D', 0

In [114]:
print(f"base score of the topic_arg: {bag.arguments[topic_a].initial_weight}")
# sum of all attributions
total_sum = sum(attribution.values())
print(f"sum of all attributions: {total_sum}")
print("--------------------------------------------")
print(f"sum of all attributions + BS: {total_sum+bag.arguments[topic_a].initial_weight}")
print(f"strength of the topic_arg: {bag.arguments[topic_a].strength}")

base score of the topic_arg: 0.79
sum of all attributions: 0.060828523299999986
--------------------------------------------
sum of all attributions + BS: 0.8508285233
strength of the topic_arg: 0.84774307
